In [ ]:
! pip install dnspython

     |████████████████████████████████| 245kB 5.2MB/s 


In [ ]:
import pymongo
from pymongo import MongoClient
# Connecting to your client MongoDB


#airbnb is the collection
airbnb=client['sample_airbnb']['listingsAndReviews']
geospatial=client['sample_geospatial']['shipwrecks']
movies=client['sample_mflix']['movies']
supplies=client['sample_supplies']['sales']
training=client['sample_training']
weather=client['sample_weatherdata']['data']
grades = training['grades'] 

# OTHER NOTES --- 
# list dbs
# list_databases()

# list collections
# list_collection_names()

# SORT --> SKIP --> LIMIT
# collection.distinct('address.country')

In [ ]:
list(airbnb.find({'bedrooms': 10}, {"_id":0, "name":1, "bedrooms":1}))

[{'bedrooms': 10, 'name': 'Great Complex of the Cellars'},
 {'bedrooms': 10, 'name': 'Excellent Location and price !'}]

### All Query Operators discussed below can be found here:
https://docs.mongodb.com/manual/reference/operator/query/

## Question 1  

Find the number of listings in the Airbnb collection with "accommodates" value of 5 i.e. that accomodates 5 people. 

In [ ]:
# The curly braces are for the filter arg 
airbnb.count_documents({"accommodates":5})

264

In [ ]:
# Most read operations begin with the collection.find
airbnb.find({"accommodates":5}).count()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


264

## Question 2 

Find the number of documents in the Airbnb collection that have a "TV" as a value in the "amenities" array.

In [ ]:
# Fields are case sensitive, so watch out when you're accessing them
airbnb.count_documents({"amenities":"TV"})

#If you did airbnb.count_documents({"amenities":["TV"]}), then it's asking for "do you find an array inside amenities and inside that array do you find TV"

4280

## Question 3

Find all the listings with 10 bedrooms, sorted by name descending and ONLY return the name, bedrooms, amenities, and address field. Limit to 1 result.

SQL code will look like:



```
SELECT name, bedrooms, amenities, address field
FROM airbnb
WHERE bedrooms=10
ORDER BY 1 DESC
LIMIT 1
```



In [ ]:
list(
    airbnb
     .find({"bedrooms":10}, {"name":1,"bedrooms":1,"amenities":1, "address":1,"_id":0 })
     .sort("name",-1)
     .limit(1)    
)
# One curly braces for filter, another for projection/what you're returning
#"_id" is the primary key. It will always return the primary key
# Projection is on a binary scale - so 1 tells the server to project the field in the result, 0 tells it not to project the field on the output
# Sort's -1 implies descending order, 1 for ascending

[{'address': {'country': 'Portugal',
   'country_code': 'PT',
   'government_area': 'Santa Marinha e São Pedro da Afurada',
   'location': {'coordinates': [-8.62181, 41.13354],
    'is_location_exact': False,
    'type': 'Point'},
   'market': 'Porto',
   'street': 'Vila Nova de Gaia, Porto, Portugal',
   'suburb': ''},
  'amenities': ['TV',
   'Cable TV',
   'Wifi',
   'Kitchen',
   'Free street parking',
   'Heating',
   'Family/kid friendly',
   'Washer',
   'Dryer',
   'First aid kit',
   'Fire extinguisher',
   'Essentials',
   'Shampoo',
   'Lock on bedroom door',
   'Hangers',
   'Hair dryer',
   'Iron',
   'Laptop friendly workspace',
   'Private entrance',
   'Baby monitor',
   'Bathtub',
   'Baby bath',
   'High chair',
   'Crib',
   'Pack ’n Play/travel crib',
   'Hot water',
   'Bed linens',
   'Extra pillows and blankets',
   'Microwave',
   'Coffee maker',
   'Refrigerator',
   'Dishes and silverware',
   'Cooking basics',
   'Oven',
   'Stove',
   'BBQ grill',
   'Patio 

In [ ]:
airbnb.find({"bedrooms":10}, {"name":1,"bedrooms":1,"amenities":1, "address":1,"_id":0 }).sort("name",-1).limit(1)  

# To avoid the below result - always use a LIST! 


## Question 4

SELECT name, number_of_reviews  <br>
FROM airbnb <br>
ORDER BY number_of_reviews desc <br>
LIMIT 1

In [ ]:
list(
    airbnb.find({},{"name":1,"number_of_reviews":1,"_id":0})
    .sort("number_of_reviews",-1)
    .limit(1)
)
# You still need the empty {} for the filter criteria

[{'name': '#Private Studio - Waikiki Dream', 'number_of_reviews': 533}]

# Comparison Operators

~~~
$eq: Matches values that are equal to a specified value.
$gt: Matches values that are greater than a specified value.
$gte: Matches values that are greater than or equal to a specified value.
$in: Matches any of the values specified in an array.
$lt: Matches values that are less than a specified value.
$lte: Matches values that are less than or equal to a specified value.
$ne: Matches all values that are not equal to a specified value.
$nin: Matches none of the values specified in an array.
~~~

See https://docs.mongodb.com/manual/reference/operator/query-comparison/ for more info.

## Question 5 

Return the document where "accommodates" is greater than 6, and ordered by "number_of_reviews" descending, limit to 1 result. Only return "name", "accommodates", "number_of_reviews", and the "address".

In [ ]:
list(
    airbnb
     .find(
         {"accommodates":{"$gt":6}}, # The inner curly braces tells Mongo that there's a logical operation and it's not a simple retrieve
         {"name":1, "accommodates":1, "number_of_reviews":1, "address":1,"_id":0}
     )
     .sort("number_of_reviews",-1)
     .limit(1) 

)

[{'accommodates': 11,
  'address': {'country': 'Spain',
   'country_code': 'ES',
   'government_area': 'el Putxet i el Farró',
   'location': {'coordinates': [2.1465, 41.40914],
    'is_location_exact': False,
    'type': 'Point'},
   'market': 'Barcelona',
   'street': 'Barcelona, Catalonia, Spain',
   'suburb': 'Sarrià-Sant Gervasi'},
  'name': 'Spectacular Modern Uptown Duplex',
  'number_of_reviews': 273}]

## Question 6 - $nin

Return the document where the "accommodates" is greater than 3 and less than 6, and "amenities" value where that does not contain a "TV" nor "Internet" and "number_of_reviews" greater than or equal to 22. Only return the name, accommodates, amenities and number_of_reviews fields and sort by the number_of_reviews field descending. Limit to 1 result.  

In [ ]:
list(
    airbnb
     .find(
       {"accommodates":{"$gt":3,"$lt":6},
        "amenities":{"$nin":["TV","Internet"]},
        "number_of_reviews": {"$gte":22}
        },
       { "name":1, "accommodates":1, "amenities":1, "number_of_reviews":1,"_id":0 })
     .sort("number_of_reviews",-1).limit(1)
)

[{'accommodates': 5,
  'amenities': ['Wifi',
   'Kitchen',
   'Free parking on premises',
   'Buzzer/wireless intercom',
   'Family/kid friendly',
   'Washer',
   'First aid kit',
   'Fire extinguisher',
   'Essentials',
   'Shampoo',
   '24-hour check-in',
   'Hangers',
   'Hair dryer',
   'Iron',
   'Laptop friendly workspace',
   'translation missing: en.hosting_amenity_50',
   'High chair',
   'Hot water',
   'Bed linens',
   'Ethernet connection',
   'Microwave',
   'Coffee maker',
   'Refrigerator',
   'Dishes and silverware',
   'Cooking basics',
   'Oven',
   'Stove',
   'Patio or balcony',
   'Luggage dropoff allowed',
   'Long term stays allowed',
   'Cleaning before checkout',
   'Wide hallway clearance',
   'Wide doorway',
   'Flat path to front door',
   'Well-lit path to entrance',
   'Step-free access',
   'Accessible-height bed',
   'Step-free access',
   'Accessible-height toilet',
   'Host greets you'],
  'name': 'Lovely apartment with private parking',
  'number_of_r

## Question 7 - Special Case $nin

Return the count of documents that do not have any of the following cast members: "Gèrard Depardieu", "Robert De Niro", "Morgan Freeman", "Samuel L. Jackson","Nicolas Cage", nor "Bruce Willis". 


In [ ]:
#movies["movies"] and the field is "cast"


movies.movies.count_documents({
"cast":{"$nin" : ["Gèrard Depardieu", "Robert De Niro", "Morgan Freeman", "Samuel L. Jackson","Nicolas Cage", "Bruce Willis"]}
        })


23217

In [ ]:

original = movies.movies.count_documents({
"cast":{"$nin" : ["Gèrard Depardieu", "Robert De Niro", "Morgan Freeman", "Samuel L. Jackson","Nicolas Cage", "Bruce Willis"]}
        })
castExists=movies.movies.count_documents({
"cast":{"$nin" : ["Gèrard Depardieu", "Robert De Niro", "Morgan Freeman", "Samuel L. Jackson","Nicolas Cage", "Bruce Willis"],
        "$exists":True}
        })
noCast=movies.movies.count_documents({
"cast":{"$exists":False}}
        )
print(f'original: {original}, cast exists: {castExists}, cast doesnt exist: {noCast}')

original: 23217, cast exists: 22827, cast doesnt exist: 390


## Question 8

Write the following in MongoDB:
    
SELECT name, number_of_reviews, beds <br>
FROM airbnb <br>
WHERE number_of_reviews >= 200 <br>
ORDER BY beds DESC <br>
LIMIT 1

In [ ]:
list(
    airbnb.find({"number_of_reviews":{"$gte":200}},
                {"name":1,"number_of_reviews":1,"beds":1,"_id":0}
                ).sort("beds",-1).limit(1)
)

[{'beds': 7,
  'name': 'Beautiful Home Away From Home!',
  'number_of_reviews': 219}]

## Question 9 - Special Case $ne

Return the count of all documents where the "tomatoes.viewer.rating" value is "$ne" to 3.

In [ ]:
movies.count_documents({"tomatoes.viewer.rating":{"$ne":3}})

22492

In [ ]:
movies.count_documents({"tomatoes.viewer.rating":{"$ne":3,"$exists":True}})

19518

## Question 10 - $in

Return the count of all the documents that have the "property_type" value of "Bungalow" and "amenities" that have the following items: "TV", "Wifi","Smart TV", "Heating", "Swimming Pool", "Waterfront","Gym","Beachfront","Safe","Refrigerator", and a "Tennis Court".


In [ ]:
airbnb.count_documents({"property_type":"Bungalow",
                        "amenities":{"$in":["TV", "Wifi","Smart TV", "Heating", "Swimming Pool", "Waterfront","Gym","Beachfront","Safe","Refrigerator","Tennis Court"]}})

14

# Logical Operators

~~~
$and: Joins query clauses with a logical AND returns all documents that match the conditions of both clauses.
$not: Inverts the effect of a query expression and returns documents that do not match the query expression.
$nor: Joins query clauses with a logical NOR returns all documents that fail to match both clauses.
$or: Joins query clauses with a logical OR returns all documents that match the conditions of either clause.
~~~

See https://docs.mongodb.com/manual/reference/operator/query-logical/ for more info.

OR
- diff fields
- multiples expressions

IN
- one field, multiple values

```

```



In [ ]:
# format: 
#{$or:[{expression 1}, {expression 2}...]}

## Question 11

Rewrite the above question (Question 10) using the $or operator. 

In [ ]:
airbnb.count_documents({"property_type":"Bungalow",
                        "$or":[
                               {"amenities":"TV"},{"amenities": "Wifi"},{"amenities":"Smart TV"},{"amenities": "Heating"}
                               , {"amenities":"Swimming Pool"}, {"amenities":"Waterfront"},\
                                {"amenities":"Gym"},{"amenities":"Beachfront"},{"amenities":"Safe"}
                               ,{"amenities":"Refrigerator"},{"amenities":"Tennis Court"}]})

14

## Question 12

Return the count of all documents in the movies database and movies collection (i.e. movies["movies"]) where the document had either a "rated" value of "PG" or "PG-13".

In [ ]:
movies.count_documents(
    {"rated":{"$in":["PG","PG-13"]}}
)

4173

## Question 13 - conjunctive

Return two movies sorted by year descending where either the "awards.wins" value is greater than 250 or 
the "year" is between 2000 and 2005.

In [ ]:
list(
    movies.find({
        "$or":[
        {"awards.wins":{"$gt":250}},
        {"year":{"$gte":2000,"$lte":2005}}]}, # There is an implicit AND here which is not in the next question
        {"title":1,"year":1,"awards.wins":1,"_id":0}
    ).sort("year",-1).limit(2)
)

# We get 2013 because of our OR clause - either nore than 250 wins or between the year 2000 and 2005. So 12 years a slave still makes it.

[{'awards': {'wins': 267}, 'title': '12 Years a Slave', 'year': 2013},
 {'awards': {'wins': 1}, 'title': 'Fantastic Four', 'year': 2005}]

Complex Sort:

```.sort([(field1, ASC),(field2, DESC),(...),(...)])```

## Question 14 - disjunctive

Return 3 movies sorted by year descending where either the "awards.wins" value is greater than 250 or 
the "year" is less than 2005 or greater than 2015.

In [ ]:
list(
    movies.find({
        "$or":[
        {"awards.wins":{"$gt":250}},
        {"year":{"$gt":2015}},{"year":{"$lt":2005}}]},
        {"title":1,"year":1,"awards.wins":1,"_id":0}
    ).sort([("year",-1),("title",1)]).limit(3)
)


[{'awards': {'wins': 1}, 'title': 'The Masked Saint', 'year': 2016},
 {'awards': {'wins': 267}, 'title': '12 Years a Slave', 'year': 2013},
 {'awards': {'wins': 3}, 'title': '06/05', 'year': 2004}]

## Question 15 

Return a movie that has a "runtime" of greater than 90, a year greater than 2000, 
and an "imdb.votes" less than 40.

In [ ]:
list(
    movies.find(
        {
            "$and":[
                    {"runtime":{"$gt":90}},
                    {"year":{"$gt":2000}},
                    {"imdb.votes":{"$lt":40}}

            ]
        }
    ).sort("year",1)
    .limit(1)
)

[{'_id': ObjectId('573a13a9f29313caabd1e696'),
  'awards': {'nominations': 0, 'text': '1 win.', 'wins': 1},
  'cast': ['Joe Anderson', 'Rachel Finan', 'Jeff Heilman', 'Liz Hinkley'],
  'countries': ['USA'],
  'directors': ['Joe Anderson'],
  'fullplot': "One family's paranoid preparation for the year 2000 crisis and beyond. Survivalist Carl Foster has one year to convince his family he's not crazy and independent filmmaker Atis Jamison has 365 days to get it all on tape.",
  'genres': ['Comedy'],
  'imdb': {'id': 314325, 'rating': 7.9, 'votes': 10},
  'languages': ['English'],
  'lastupdated': '2015-09-02 00:23:45.257000000',
  'num_mflix_comments': 2,
  'plot': "One family's paranoid preparation for the year 2000 crisis and beyond. Survivalist Carl Foster has one year to convince his family he's not crazy and independent filmmaker Atis Jamison has ...",
  'released': datetime.datetime(2002, 3, 24, 0, 0),
  'runtime': 113,
  'title': 'Living with the Fosters',
  'tomatoes': {'lastUpdat

## Question 16 - review several ways of $and

Return the document's title sorted by "year" descending that has a "runtime" value greater than 100, "year" greater than 2000 and less than 2005, and an "imdb.votes" value less than 30. Show 3 different ways of writing this query.  

In [ ]:
list(
    movies.find({ "$and":[
                          {"runtime":{"$gt":100}},
                          {"year":{"$gt":2000}},
                          {"year":{"$lt":2005}},
                          {"imdb.votes":{"$lt":30}}
    ]
        
    },
    {"title":1,"_id":0}).sort("year",-1).limit(1)
)

[{'title': 'Living with the Fosters'}]

In [ ]:
list(
    movies.find(
        {"runtime":{"$gt":100},
         "year":{"$gt":2000},
         "year":{"$lt":2005},
         "imdb.votes":{"$lt":30}},
        {"title":1,"_id":0}
    )
    .sort("year",-1).limit(1)
)

[{'title': 'Living with the Fosters'}]

In [ ]:
list(
    movies.find(
        {"runtime":{"$gt":100},
         "year":{"$gt":2000,"$lt":2005},
         "imdb.votes":{"$lt":30}},
        {"title":1,"_id":0}
    )
    .sort("year",-1).limit(1)
)

[{'title': 'Living with the Fosters'}]

## Question 17 - implicit vs explicit $and


Return a document that has a "type" of movie, and where the "year" value is less than 1930 or rated a "PG", and the "imdb.votes" is greater than 100 or the runtime is greater than 120.

In [ ]:
list(
    movies.find(
        {
          "$and":[
          {"type":{"$exists":True}},
           {"$or":[
                 {"year":{"$lt":1930}},
                 {"rating":"PG"}
         ]},
         {"$or":[
         {"imdb.votes":{"$gt":100}},
         {"runtime":{"$gt":120}}]}
          ]},
    ).sort("year",1).limit(1)
)

# You need an explicit AND for the OR to be considered

[{'_id': ObjectId('573a13a3f29313caabd0d5a4'),
  'awards': {'nominations': 0, 'text': '1 win.', 'wins': 1},
  'countries': ['USA'],
  'directors': ['William K.L. Dickson'],
  'fullplot': 'A young man stands before the camera holding a club in each hand, horizontal to the ground. He raises the heads of the two clubs in unison, by rotating the clubs without lifting his arms. The film then shows the same footage over again, at different speeds.',
  'genres': ['Documentary', 'Short'],
  'imdb': {'id': 241763, 'rating': 4.9, 'votes': 827},
  'languages': ['English'],
  'lastupdated': '2015-08-03 00:57:26.680000000',
  'num_mflix_comments': 3,
  'plot': 'An athlete swings Indian clubs.',
  'rated': 'NOT RATED',
  'runtime': 1,
  'title': 'Newark Athlete',
  'tomatoes': {'lastUpdated': datetime.datetime(2012, 9, 30, 0, 0)},
  'type': 'movie',
  'year': 1891}]

## Question 18

Return the count of all the documents in training['grades'] dataset where none of the "scores.score" values are less than 93.5

In [ ]:
training["grades"].find_one({})
# What one document looks like

{'_id': ObjectId('56d5f7eb604eb380b0d8d8e1'),
 'class_id': 460.0,
 'scores': [{'score': 33.63395086942907, 'type': 'exam'},
  {'score': 46.7200034427161, 'type': 'quiz'},
  {'score': 13.084793373731118, 'type': 'homework'},
  {'score': 30.05232124487215, 'type': 'homework'}],
 'student_id': 1.0}

In [ ]:
training["grades"].count_documents({
         "scores.score":{"$not":{"$lt":93.5}}
     })

# We can't use $gt 93.5 since $gt looks at only one value. If one score passes, it will return that document but there are a number of other score entries
# in the same field. For your filter to apply to all values in the field, but 

2



# Other Operators

Element, Evaluation, Geospatial, and Array operators

~~~
$text: Performs text search.
{
  $text:
    {
      $search: <string>,
      $language: <string>,
      $caseSensitive: <boolean>,
      $diacriticSensitive: <boolean>
    }
}


$elemMatch: Selects documents if element in the array field matches all the specified $elemMatch conditions.
{ <field>: { $elemMatch: { <query1>, <query2>, ... } } }



$size: Selects documents if the array field is a specified size.
{ <field>: { $size: value } }



$all: Matches arrays that contain all elements specified in the query.
{ <field>: { $all: [ <value1> , <value2> ... ] } }

~~~
text - https://docs.mongodb.com/manual/reference/operator/query/text <br>
elemMatch - https://docs.mongodb.com/manual/reference/operator/query/elemMatch/ <br>
size - https://docs.mongodb.com/manual/reference/operator/query/size/ <br>
all - https://docs.mongodb.com/manual/reference/operator/query/all/

## Question 19 - $text

Return the documents sorted by beds descending that have the word "park" or the phrase "los angeles" in the description field (index has already been created on "description") in the Airbnb dataset. Only return the description, name and limit to 1 result sorted by "beds" descending.

## Question 20 - $elemMatch

In the geospatial dataset, sorted by "latdec" descending, find a document where the "coordinates" the longitude value is greater than -90 but less than or equal -88. Limit to 1 result. 

In [ ]:
list(
    geospatial
     .find({
         "coordinates":{
             "$elemMatch":{
                 "$lte":-88,"$gt":-90
             }
         }
    }).sort("latdec",-1).limit(1)
)

[{'_id': ObjectId('578f6fa3df35c7fbdbaee7ef'),
  'chart': 'US,US,graph,chart 14976',
  'coordinates': [-88.49157, 48.200457],
  'depth': '',
  'feature_type': 'Wrecks - Submerged, dangerous',
  'gp_quality': '',
  'history': '',
  'latdec': 48.200457,
  'londec': -88.49157,
  'quasou': 'depth unknown',
  'recrd': '',
  'sounding_type': '',
  'vesslterms': '',
  'watlev': 'always under water/submerged'}]

In [ ]:
list(
    geospatial
     .find({
         "coordinates":{"$lte":-88,"$gt":-90} #this is like a BETWEEN, not the same as above
    }).sort("latdec",-1).limit(1)
)

[{'_id': ObjectId('578f6fa3df35c7fbdbaed9d7'),
  'chart': 'US,US,reprt,17thCGD,LNM\xa045/09',
  'coordinates': [-148.516786, 70.3858306],
  'depth': '',
  'feature_type': 'Wrecks - Visible',
  'gp_quality': '',
  'history': '',
  'latdec': 70.3858306,
  'londec': -148.516786,
  'quasou': '',
  'recrd': '',
  'sounding_type': '',
  'vesslterms': 'West Dock Barge',
  'watlev': 'always dry'}]

In [ ]:
# If i dont want any values in my coord array < than -90
list(
    geospatial
     .find({
         "coordinates":{
             "$not":{"$elemMatch"......}
             }
         }
    }).sort("latdec",-1).limit(1)
)

## Question 21 - $elemMatch

In the training["companies"] dataset, return the count of documents that have at least one embedded document in the "relationships" field where the "person.first_name" value is "Ben" and the "is_past" value is False. Compare this count with the count of documents where the "relationships" field has an "is_past" value of false and a "person.first_name" value of "Ben".

In [ ]:
elemMatch=training.companies.count_documents({
    "relationships":{
        "$elemMatch":{
            "is_past":False,
            "person.first_name":"Ben"
        }
    }
})
noMatch=training.companies.count_documents({
    "relationships.is_past":False,
    "relationships.person.first_name":"Ben"
})

print(f'elemMatch:{elemMatch},noMatch:{noMatch}')

elemMatch:93,noMatch:167


## Question 22 - $elemMatch

In the grades collection, find a document where one element matches the type of "scores.type" 
of "exam" and "scores.score" is greater than 97.

In [ ]:
list(
    grades.find({
      "scores":{
          "$elemMatch":{
              "type":"exam",
              "score":{"$gt":97}
          }
      }  
    }).sort("student_id",-1)
    .limit(1)
)

# In the o/p, we see that within a single element in scores field, a single entry has both exam and >97
# This is the right answer to the question

[{'_id': ObjectId('56d5f7f1604eb380b0da5f39'),
  'class_id': 379.0,
  'scores': [{'score': 98.20044944087331, 'type': 'exam'},
   {'score': 28.349052884591032, 'type': 'quiz'},
   {'score': 69.8395825261821, 'type': 'homework'},
   {'score': 21.288484910376116, 'type': 'homework'}],
  'student_id': 9994.0}]

In [ ]:
list(
    grades.find({
      "scores.type":"exam",
      "scores.score":{"$gt":97}
    }).sort("student_id",-1)
    .limit(1)
)
# This o/p returns a document where any element in the entire array meets the two conditions.
# One element within the array meets the 97 criteria and another element in the array/subdocuments meets the exam criteria
# This passes the document for the condition

[{'_id': ObjectId('56d5f7f1604eb380b0da5f4f'),
  'class_id': 418.0,
  'scores': [{'score': 22.56383221610897, 'type': 'exam'},
   {'score': 52.57100113131188, 'type': 'quiz'},
   {'score': 10.923540251153852, 'type': 'homework'},
   {'score': 98.40768422639076, 'type': 'homework'}],
  'student_id': 9996.0}]

## Question 23 - $size

In the Airbnb dataset, return the count of documents that have an "amenities" field size of 25.


In [ ]:
# Size evaluates the size of the array. Can't aggregate or filter by size here.
airbnb.count_documents({"amenities":{"$size":25}})

173

## Question 24 - $all

Return the count of documents that have all the following values in the "amenities" field: 'TV', 'Kitchen', 'Wifi', 'Internet','Cable TV', 'Coffee maker', 'Cooking basics','Family/kid friendly', 'Gym', and 'Heating'. 
\$in \$nin

In [ ]:
# You can pass in other operators with the $all
airbnb.count_documents({
    "amenities":{
        "$all":['TV', 'Kitchen', 'Wifi', 'Internet','Cable TV', 'Coffee maker', 'Cooking basics','Family/kid friendly', 'Gym', 'Heating']
    }
})
# 8 documents contain all these values. $all doesn't care about the order of elements

8

In [ ]:
airbnb.count_documents({
    "amenities":['TV', 'Kitchen', 'Wifi', 'Internet','Cable TV', 'Coffee maker', 'Cooking basics','Family/kid friendly', 'Gym', 'Heating']
    }
)

# This tells it to search for the same exact order you pass in the array. Useful if the exact order is important but usually prone to errors.

0

## Question 25 - $all with elemMatch

Return a single document sorted by class_id descending where there's one embeddded document that has "scores.type" of exam and "scores.score" of greater than 97, "scores.type" of quiz and "scores.score" of greater than 96, and
"scores.type" of homework and "scores.score" of greater than 95.

In [ ]:
list(
    grades.find({
        "scores":{
            "$all":[
                    {"$elemMatch":{
                        "type":"exam","score":{"$gt":97}
                    }},
                    {"$elemMatch":{
                        "type":"quiz","score":{"$gt":96}
                        }},
                    {"$elemMatch":{
                        "type":"homework","score":{"$gt":95}
                        }}
            ]
        }
    }).sort("class_id",-1)
    .limit(1)
)

# All ensures there's 1 instance of everything in the output
# You still get a type: homework with score:43 but the next element in the array meets the condition.

[{'_id': ObjectId('56d5f7f1604eb380b0da4b05'),
  'class_id': 439.0,
  'scores': [{'score': 97.70221871186124, 'type': 'exam'},
   {'score': 98.38639086583655, 'type': 'quiz'},
   {'score': 43.71678839949483, 'type': 'homework'},
   {'score': 97.78339211813662, 'type': 'homework'}],
  'student_id': 9477.0}]

In [ ]:
list(
    grades.find({
        "scores":{
            
                    "$elemMatch":{
                        "type":"exam","score":{"$gt":97}},
                    "$elemMatch":{
                        "type":"quiz","score":{"$gt":96}
                        },
                    "$elemMatch":{
                        "type":"homework","score":{"$gt":95}
                        }
            
        
    }
    }).sort("class_id",-1)
    .limit(1)
)

# Incorrrect!

[{'_id': ObjectId('56d5f7eb604eb380b0d8da7e'),
  'class_id': 500.0,
  'scores': [{'score': 29.438528115173668, 'type': 'exam'},
   {'score': 93.45862647470958, 'type': 'quiz'},
   {'score': 95.83097600441303, 'type': 'homework'},
   {'score': 96.56935104413706, 'type': 'homework'}],
  'student_id': 43.0}]